In [0]:
import numpy as np
import sys
import os
import random
from skimage import io
import pandas as pd
import cv2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Load data from .csv file

* `train-images-boxable.csv` file contains the image name and image url
* `train-annotations-bbox.csv` file contains the bounding box info with the image id (name) and the image label name
* `class-descriptions-boxable.csv` file contains the image label name corresponding to its class name

Download link:

https://storage.googleapis.com/openimages/web/download.html (Download from Figure Eight)

https://www.figure-eight.com/dataset/open-images-annotated-with-bounding-boxes/

# Parameter

In [0]:
n = 400
label_list = ['Person','Car']
base_path = '/content/drive/My Drive/computer vision/Data_processing'
export_path = '/content/drive/My Drive/computer vision/Data_processing'
images_boxable_fname = 'train-images-boxable.csv'
annotations_bbox_fname = 'train-annotations-bbox.csv'
class_descriptions_fname = 'class-descriptions-boxable.csv'

# Common function

In [0]:
# get subset of the whole subset
def getSubsetImage(img_id):
  # Shuffle the data
  copy_img_id = img_id.copy()
  random.seed(1)
  random.shuffle(copy_img_id)

  # Extract n images, this part cost time
  sub_img_id = copy_img_id[:n]
  return [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in sub_img_id]

# save Image_url to csv files
def saveImageURL(label, sub_img_url):
  sub_pd = pd.DataFrame()
  for i in range(len(sub_img_url)):
      sub_pd = sub_pd.append(sub_img_url[i], ignore_index = True)

  sub_pd.to_csv(os.path.join(export_path, 'sub'+label+'_img_url.csv'))
  return [url['image_url'].values[0] for url in sub_img_url]

# download images to the folder
def downloadImages(sub_img_url, saved_dir):
  os.mkdir(saved_dir)
  for url in sub_img_url:
    img = io.imread(url)
    saved_path = os.path.join(saved_dir, url[-20:])
    io.imsave(saved_path, img)

In [0]:
images_boxable = pd.read_csv(os.path.join(base_path, images_boxable_fname))
images_boxable.head()

,image_name,image_url
0,e39871fd9fd74f55.jpg,https://requestor-proxy.figure-eight.com/figur...
1,f18b91585c4d3f3e.jpg,https://requestor-proxy.figure-eight.com/figur...
2,ede6e66b2fb59aab.jpg,https://requestor-proxy.figure-eight.com/figur...
3,ed600d57fcee4f94.jpg,https://requestor-proxy.figure-eight.com/figur...
4,ff47e649b23f446d.jpg,https://requestor-proxy.figure-eight.com/figur...


In [0]:
annotations_bbox = pd.read_csv(os.path.join(base_path, annotations_bbox_fname))
annotations_bbox.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


In [0]:
class_descriptions = pd.read_csv(os.path.join(base_path, class_descriptions_fname))
class_descriptions.head()

,name,class
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


# Image data extraction

In [0]:
for label in label_list:
  print("Extract image: "+ label)
  # Find the label_name for the label class
  dataset_pd = class_descriptions[class_descriptions['class']==label]
  label_name = dataset_pd['name'].values[0]

  # Find the number of images in the dataset
  dataset_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name]
  print('There are %d %s in the dataset.' %(len(dataset_bbox), label))
  img_id = np.unique(dataset_bbox['ImageID'])
  print('There are %d images which contain %s.' % (len(img_id), label))

  # Get image subset
  sub_img_url = getSubsetImage(img_id)
  print('Getting subset complete.')

  # save image_url data to csv 
  sub_img_url = saveImageURL(label, sub_img_url)
  print("URL saved to csv file.")

  # Download images
  saved_dir = os.path.join(export_path,label)
  downloadImages(sub_img_url, saved_dir)
  print("Download complete.")
  print()

Extract image: Person
There are 1034721 Person in the dataset.
There are 248384 images which contain Person.
Getting subset complete.
URL saved to csv file.
Download complete.

Extract image: Car
There are 248075 Car in the dataset.
There are 89465 images which contain Car.
Getting subset complete.
URL saved to csv file.
Download complete.

